### Aplicação de Regressão logística para análise de variáveis e sua influência na eleição de candidatos em eleições no Brasil

##### Foram utilizadas informações das eleições federais de 2014 e 2018 disponíveis no site do Tribunal Superior Eleitoral: http://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1/repositorio-de-dados-eleitorais

In [ ]:
# Importação de pacotes utilizados na análise
import pandas as pd
import numpy as py
import matplotlib.pyplot as plt
import seaborn as sns

#### 1. Preparação das bases

##### 1.1 Importação dos arquivos usados na análise

In [ ]:
# Candidatos
cand_2014 = pd.read_csv('consulta_cand_2014_BRASIL.csv', sep=';', encoding='latin1', low_memory=False)
cand_2018 = pd.read_csv('consulta_cand_2018_BRASIL.csv', sep=';', encoding='latin1', low_memory=False)

# Patrimônio
bem_cand_2014 = pd.read_csv('bem_candidato_2014_BRASIL.csv', sep=';', encoding='latin1', low_memory=False)
bem_cand_2018 = pd.read_csv('bem_candidato_2018_BRASIL.csv', sep=';', encoding='latin1', low_memory=False)

# Despesas de campanha
despesas_2014 = pd.read_csv('despesas_candidatos_2014_brasil.txt', sep=';', encoding='latin1', low_memory=False)
despesas_2018 = pd.read_csv('despesas_contratadas_candidatos_2018_BRASIL.csv', sep=';', encoding='latin1', low_memory=False)

##### 1.2 Verificação de dados básicos acerca das bases  importadas

In [ ]:
print('Dimensões - Candidatos 2014: ', cand_2014.shape)
print('Dimensões - Candidatos 2018: ', cand_2018.shape)
print('Dimensões - Bens candidatos 2014: ', bem_cand_2014.shape)
print('Dimensões - Bens candidatos 2018: ', bem_cand_2018.shape)
print('Dimensões - Despesas Candidatos 2014: ', despesas_2014.shape)
print('Dimensões - Despesas Candidatos 2018: ', despesas_2018.shape)

In [ ]:
cand_2014.info()

In [ ]:
cand_2014.head()

In [ ]:
cand_2018.info()

In [ ]:
cand_2018.head()

In [ ]:
bem_cand_2014.info()

In [ ]:
bem_cand_2014.head()

In [ ]:
bem_cand_2018.info()

In [ ]:
bem_cand_2018.head()

In [ ]:
despesas_2014.info()

In [ ]:
despesas_2014.head()

In [ ]:
despesas_2018.info()

In [ ]:
despesas_2018.head()

##### 1.3 Base Candidatos

In [ ]:
# Para a análise serão mantidas apenas as colunas consideradas úteis ao estudo. 

cand_2014.drop(columns=['DT_GERACAO', 'HH_GERACAO', 'CD_TIPO_ELEICAO', 'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 
                        'DS_ELEICAO', 'DT_ELEICAO', 'SG_UE', 'NR_CANDIDATO', 'NM_URNA_CANDIDATO', 'NM_SOCIAL_CANDIDATO',
                        'NM_EMAIL', 'TP_AGREMIACAO', 'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO', 'SQ_COLIGACAO', 'NM_COLIGACAO', 
                        'DS_COMPOSICAO_COLIGACAO', 'SG_UF_NASCIMENTO', 'CD_MUNICIPIO_NASCIMENTO', 'NM_MUNICIPIO_NASCIMENTO', 
                        'DT_NASCIMENTO', 'NR_TITULO_ELEITORAL_CANDIDATO', 'NR_DESPESA_MAX_CAMPANHA', 'ST_DECLARAR_BENS',
                        'NR_PROTOCOLO_CANDIDATURA', 'NR_PROCESSO'],inplace=True) 

cand_2018.drop(columns=['DT_GERACAO', 'HH_GERACAO', 'CD_TIPO_ELEICAO', 'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 
                        'DS_ELEICAO', 'DT_ELEICAO', 'SG_UE', 'NR_CANDIDATO', 'NM_URNA_CANDIDATO', 'NM_SOCIAL_CANDIDATO',
                        'NM_EMAIL', 'TP_AGREMIACAO', 'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO', 'SQ_COLIGACAO', 'NM_COLIGACAO', 
                        'DS_COMPOSICAO_COLIGACAO', 'SG_UF_NASCIMENTO', 'CD_MUNICIPIO_NASCIMENTO', 'NM_MUNICIPIO_NASCIMENTO', 
                        'DT_NASCIMENTO', 'NR_TITULO_ELEITORAL_CANDIDATO', 'NR_DESPESA_MAX_CAMPANHA', 'ST_DECLARAR_BENS',
                        'NR_PROTOCOLO_CANDIDATURA', 'NR_PROCESSO'],inplace=True) 

In [ ]:
cand_2014.columns

In [ ]:
cand_2018.columns

In [ ]:
# Na base de dados, candidatos que vão para o segundo turno são registrados duas vezes. É necessário excluir este registro afim 
# de se evitar duplicidades, inconsistências (CD_SIT_TOT_TURNO = 6)

cand_2014 = cand_2014[cand_2014.CD_SIT_TOT_TURNO != 6]
cand_2018 = cand_2018[cand_2018.CD_SIT_TOT_TURNO != 6]


# Outros filtros podem ser aplicados aqui

##### 1.4 Base Bens de candidatos

In [ ]:
# Formatação dos dados do campo VR_BEM_CANDIDATO. Os dados em formato numérico nacional podem apresentar conflitos com o 
# padrão americano.
bem_cand_2014['VR_BEM_CANDIDATO'] = bem_cand_2014['VR_BEM_CANDIDATO'].str.replace(',','.').astype(float)

In [ ]:
bem_cand_2018['VR_BEM_CANDIDATO'] = bem_cand_2018['VR_BEM_CANDIDATO'].str.replace(',','.').astype(float)

In [ ]:
# Agregação dos bens dos candidatos por SQ_CANDIDATO
bem_2014 = bem_cand_2014.groupby('SQ_CANDIDATO').sum()
bem_2014.drop(columns=['ANO_ELEICAO','CD_TIPO_ELEICAO','CD_ELEICAO','NR_ORDEM_CANDIDATO','CD_TIPO_BEM_CANDIDATO'],inplace=True)

In [ ]:
bem_2018 = bem_cand_2018.groupby('SQ_CANDIDATO').sum()
bem_2018.drop(columns=['ANO_ELEICAO','CD_TIPO_ELEICAO','CD_ELEICAO','NR_ORDEM_CANDIDATO','CD_TIPO_BEM_CANDIDATO'],inplace=True)

In [ ]:
bem_2014.head()

In [ ]:
bem_2018.head()

##### 1.5 Base despesas de campanha

In [ ]:
# Formatação dos dados do campo VR_BEM_CANDIDATO. Os dados em formato numérico nacional podem apresentar conflitos com o 
# padrão americano.
despesas_2014['Valor despesa'] = despesas_2014['Valor despesa'].str.replace(',','.').astype(float)

In [ ]:
despesas_2018['VR_DESPESA_CONTRATADA'] = despesas_2018['VR_DESPESA_CONTRATADA'].str.replace(',','.').astype(float)

In [ ]:
# Ajustes dos nomes da base 2014
novo_nome_coluna = ['Cód. Eleição', 'Desc. Eleição', 'Data e hora', 'CNPJ Prestador Conta', 'SQ_CANDIDATO', 'UF', 
                    'Sigla  Partido', 'Número candidato', 'Cargo', 'Nome candidato', 'CPF do candidato', 'Tipo do documento', 
                    'Número do documento', 'CPF/CNPJ do fornecedor', 'Nome do fornecedor', 
                    'Nome do fornecedor (Receita Federal)', 'Cod setor econômico do fornecedor', 
                    'Setor econômico do fornecedor', 'Data da despesa', 'VR_DESPESA_CONTRATADA', 'Tipo despesa', 
                    'Descriçao da despesa']
despesas_2014.columns = novo_nome_coluna
despesas_2014.columns

In [ ]:
# Agrupamento das despesas por candidato
despesas_2014 = despesas_2014.groupby('SQ_CANDIDATO').sum()

In [ ]:
despesas_2018 = despesas_2018.groupby('SQ_CANDIDATO').sum()

In [ ]:
# Exclusão das colunas não utilizadas na análise
despesas_2014.drop(columns=['Cód. Eleição','CNPJ Prestador Conta', 'Número candidato', 'CPF do candidato'], inplace=True)

In [ ]:
despesas_2018.drop(columns=['ANO_ELEICAO', 'CD_TIPO_ELEICAO', 'CD_ELEICAO', 'ST_TURNO', 'SQ_PRESTADOR_CONTAS', 
                            'NR_CNPJ_PRESTADOR_CONTA', 'CD_CARGO', 'NR_CANDIDATO', 'NR_CPF_CANDIDATO', 
                            'NR_CPF_VICE_CANDIDATO', 'NR_PARTIDO', 'CD_TIPO_FORNECEDOR', 'CD_CNAE_FORNECEDOR', 
                            'NR_CPF_CNPJ_FORNECEDOR', 'CD_ESFERA_PART_FORNECEDOR', 'CD_MUNICIPIO_FORNECEDOR', 
                            'SQ_CANDIDATO_FORNECEDOR', 'NR_CANDIDATO_FORNECEDOR', 'CD_CARGO_FORNECEDOR', 
                            'NR_PARTIDO_FORNECEDOR', 'CD_ORIGEM_DESPESA', 'SQ_DESPESA'], inplace=True)

In [ ]:
despesas_2014.head()

In [ ]:
despesas_2018.head()

##### 1.6 União das bases de candidato, despesas e bem

In [ ]:
cand_2014 = pd.merge(cand_2014, bem_2014, on='SQ_CANDIDATO', how='left')
cand_2014 = pd.merge(cand_2014, despesas_2014, on='SQ_CANDIDATO', how='left')

In [ ]:
cand_2018 = pd.merge(cand_2018, bem_2018, on='SQ_CANDIDATO', how='left')
cand_2018 = pd.merge(cand_2018, despesas_2018, on='SQ_CANDIDATO', how='left')

##### 1.7 União das bases 2014 e 2018

In [ ]:
# União das duas bases
cand = cand_2014.append(cand_2018)
cand.shape

In [ ]:
# Criação da variável TARGET para análise de dados. CD_SIT_TOT_TURNO = 1 ELEITO, 2 ELEITO POR QP e 3 ELEITO POR MÉDIA

ELEITO = []

for i in cand['CD_SIT_TOT_TURNO']:
    if i == 1:
        ELEITO.append('sim')
    elif i == 2:
        ELEITO.append('sim')
    elif i == 3:
        ELEITO.append('sim')
    else:
        ELEITO.append('nao')
        
cand['ELEITO'] = ELEITO


In [ ]:
################################# A base "cand" será utilizada para a realização das análises ###############################
cand.head()

### 2. Tratamento de outliers

### 3. Análise Exploratória

### 4. Separação das bases de treino e teste (usar oversampling)

### 5. Aplicação do modelo (gstatsmodels)